In [13]:
!rm hawaii.sqlite

In [29]:
# Import our dependencies
import pandas as pd
import os
from pathlib import Path
import csv
import matplotlib.pyplot as plt
import numpy as np

In [30]:
# Dependencies and boilerplate
import sqlalchemy
from sqlalchemy import Column, Float, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine,inspect, MetaData

Base = declarative_base()

In [31]:
# Use a Session to test the Measurement class

engine = create_engine("sqlite:///hawaii.sqlite")
Base.metadata.create_all(engine)

session = Session(bind=engine)
### END SOLUTION

In [32]:
# Declare a Base using `automap_base()`
Base = automap_base()

In [33]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [34]:
# Define a Measurement class

class Measurement(Base):
    __tablename__ = "measurement"
    
    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(String)
    prcp = Column(Float)
    tobs  = Column(Integer)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [35]:
# Define a Station class

class Station(Base):
    __tablename__ = "station"
    
    id = Column(Integer, primary_key=True)
    station  = Column(String)
    name  = Column(String)
    latitude = Column(Float)
    longitude  = Column(Float)
    elevation = Column(Float)

In [36]:
# Use `create_all` to create the measurement and station table in the database
Base.metadata.create_all(engine)

In [37]:
# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)
Base.classes.keys()

[]

In [41]:
# Load the cleaned csv file into a pandas dataframe
new_m_df = pd.read_csv("/Users/nkulkar/anaconda3/clean_measurement.csv")

In [42]:
data = new_m_df.to_dict(orient='records')

In [43]:
# Data is just a list of dictionaries that represent each row of data
print(data[:5])

[{'station': 'USC00519397', 'date': '2010-01-01', 'prcp': 0.08, 'tobs': 65}, {'station': 'USC00519397', 'date': '2010-01-02', 'prcp': 0.0, 'tobs': 63}, {'station': 'USC00519397', 'date': '2010-01-03', 'prcp': 0.0, 'tobs': 74}, {'station': 'USC00519397', 'date': '2010-01-04', 'prcp': 0.0, 'tobs': 76}, {'station': 'USC00519397', 'date': '2010-01-07', 'prcp': 0.06, 'tobs': 70}]


In [44]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [45]:
# Save the reference to the `measurement` table as a variable called `table`
table = sqlalchemy.Table('measurement', metadata, autoload=True)

In [46]:
# Using `table.delete()` to remove any existing data.
conn.execute(table.delete())

In [47]:
# Use `table.insert()` to insert the data into the table
# The SQL table is populated during this step
conn.execute(table.insert(), data)

In [48]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from measurement limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0.0, 63),
 (3, 'USC00519397', '2010-01-03', 0.0, 74),
 (4, 'USC00519397', '2010-01-04', 0.0, 76),
 (5, 'USC00519397', '2010-01-07', 0.06, 70)]

In [49]:
# Save the reference to the `station` table as a variable called `table`
table = sqlalchemy.Table('station', metadata, autoload=True)

In [51]:
new_s_df =  pd.read_csv("/Users/nkulkar/anaconda3/clean_station.csv")

In [52]:
data = new_s_df.to_dict(orient='records')

In [53]:
# Data is just a list of dictionaries that represent each row of data
print(data[:5])

[{'station': 'USC00519397', 'name': 'WAIKIKI 717.2, HI US', 'latitude': 21.2716, 'longitude': -157.8168, 'elevation': 3.0}, {'station': 'USC00513117', 'name': 'KANEOHE 838.1, HI US', 'latitude': 21.4234, 'longitude': -157.8015, 'elevation': 14.6}, {'station': 'USC00514830', 'name': 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 'latitude': 21.5213, 'longitude': -157.8374, 'elevation': 7.0}, {'station': 'USC00517948', 'name': 'PEARL CITY, HI US', 'latitude': 21.3934, 'longitude': -157.9751, 'elevation': 11.9}, {'station': 'USC00518838', 'name': 'UPPER WAHIAWA 874.3, HI US', 'latitude': 21.4992, 'longitude': -158.0111, 'elevation': 306.6}]


In [54]:
# Using `table.delete()` to remove any existing data.
conn.execute(table.delete())

In [55]:
# Use `table.insert()` to insert the data into the table
# The SQL table is populated during this step
conn.execute(table.insert(), data)

In [56]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from station limit 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]